## Notka

Jak Cię interesuje tylko rezultat - otwórz plik `partie.csv`.

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [9]:
adres = "https://pl.wikipedia.org/wiki/Pos%C5%82owie_na_Sejm_Rzeczypospolitej_Polskiej_{}_kadencji"
kadencje = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII"]

In [10]:
adres.format(kadencje[0])

'https://pl.wikipedia.org/wiki/Pos%C5%82owie_na_Sejm_Rzeczypospolitej_Polskiej_I_kadencji'

In [11]:
zupa = BeautifulSoup(requests.get(adres.format(kadencje[0])).content)

/Users/user/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/user/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [12]:
tabela = zupa.find("table", "wikitable")

In [14]:
tabela.find("th").text

AttributeError: 'NoneType' object has no attribute 'find'

In [15]:
tabela.find_all("tr")[8]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [16]:
def parse_row(tr):
    tds = tr.find_all("td")
    kolor = tds[0].attrs["style"][-8:-1]
    a = tds[1].find("a")
    partia = a.text
    wikilink = a.attrs["href"]
    if partia.startswith("["):
        partia = "Niezrzeszeni"
        wikilink = ""
    return {"partia": partia, "kolor": kolor, "wikilink": wikilink}

In [17]:
[parse_row(tr) for tr in tabela.find_all("tr")[1:-1]]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [18]:
def fetch_and_parse_kadencja(kadencja):
    zupa = BeautifulSoup(requests.get(adres.format(kadencja)).content)
    tabela = zupa.find("table", "wikitable")
    naglowek = tabela.find("th").text
    if not naglowek.startswith("Ugrupowanie polityczne"):
        raise Exception("Pierwsza tabela nie zaczyna się od 'Ugrupowanie polityczne'")
    res = [parse_row(tr) for tr in tabela.find_all("tr")[1:-1]]
    return res

In [19]:
dfs = []
for i, kadencja in enumerate(kadencje):
    df = pd.DataFrame(fetch_and_parse_kadencja(kadencja))
    df["kadencja"] = i + 1
    dfs.append(df)

/Users/user/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/user/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


AttributeError: 'NoneType' object has no attribute 'find'

In [20]:
partie_i_kolor = pd.concat(dfs, ignore_index=True)
partie_i_kolor = partie_i_kolor[["kadencja", "partia", "kolor", "wikilink"]]

ValueError: No objects to concatenate

In [21]:
partie_i_kolor.head()

NameError: name 'partie_i_kolor' is not defined

In [22]:
partie_i_kolor.tail(1).iloc[0]

NameError: name 'partie_i_kolor' is not defined

In [23]:
czy_wszytkie = True
for partia, df_partii in partie_i_kolor.groupby("partia"):
    if not (df_partii["kolor"] == df_partii["kolor"].iloc[0]).all():
        print("Partia {} ma różne kolory".format(partia))
        czy_wszytkie = False
    if not (df_partii["wikilink"] == df_partii["wikilink"].iloc[0]).all():
        print("Partia {} ma różne wikilinki".format(partia))
        czy_wszytkie = False
        
if czy_wszytkie:
    print("Wszystkie partie mają kolory i wikilinki te same we wszystkich latach!")

NameError: name 'partie_i_kolor' is not defined

In [24]:
partie_i_kolor.to_csv("partie_per_kadencja.csv", index=False)

NameError: name 'partie_i_kolor' is not defined

In [25]:
partie_skrocone = partie_i_kolor.groupby("partia").first().drop("kadencja", axis=1)

NameError: name 'partie_i_kolor' is not defined

In [26]:
partie_skrocone["kadencja_pierwsza"] = partie_i_kolor.groupby("partia")["kadencja"].min()
partie_skrocone["kadencja_ostatnia"] = partie_i_kolor.groupby("partia")["kadencja"].max()

NameError: name 'partie_i_kolor' is not defined

In [27]:
len(partie_skrocone)

NameError: name 'partie_skrocone' is not defined

In [28]:
partie_skrocone.head()

NameError: name 'partie_skrocone' is not defined

In [29]:
partie_skrocone.to_csv("partie.csv")

NameError: name 'partie_skrocone' is not defined